In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:

train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
test.head()

##### Ok so there are total of 81 columns, that's hell lot of variables.

In [ ]:
train.count()

> ## Starting preliminary analysis of data

In [ ]:
train.dtypes

In [ ]:
train.describe(include = "all")

In [ ]:
train.info

#### This gave us small gist about the dataset

> ## Identifying and dealing with Missing values

In [ ]:
train.isnull()

##### I can see there are too many True values, hence missing values.

In [ ]:
#To count how many missing values are there in the dataset in row as well as column
train.isnull().sum().sum()

In [ ]:
test.isnull().sum().sum()

**##### Oh damn so there are 6965 and 7000 null values**
##### Let's see null values in each column

In [ ]:
train.isnull().sum()

##### As there are 81 columns, it's difficult to display all the columns, let's try to display only those columns which have null values.

### Now before proceeding forward, let's append train and test data so that we can deal with them together.
##### Before appending them, adding a column named 'type' to distinguish between train and  test data.

In [ ]:
#Split the train into x_train and y_train so that SalePrice can be kept separate for training later
y = train.SalePrice
X = train.drop(columns=["SalePrice"], axis=1)

In [ ]:
y.shape, X.shape, test.shape

In [ ]:
X['Type'] = 'train'
test['Type'] = 'test'
#test['SalePrice'] = -1
data = X.append(test)

In [ ]:
data.head()

In [ ]:
data.isnull().sum().sum()

**##### So now, 6965 + 7000 = 13965 null values are there in total**
##### Now as said above, let's find out specific columns which are having null values.

In [ ]:
columns_having_null_values = data[data.columns[data.isnull().sum()>0]]
columns_having_null_values

**##### Now we have a figure that 34 columns have null values out of 81 columns. This made our task much easier than before.**

## Now comes the most tidius part to deal with missing values

In [ ]:
# We have to check what values are there in the table so that we can fill values according to real world scenario.
data['Electrical'].value_counts()

##### In the above values, we can see that "Sbrkr" is the mostly used 'Electrical' part. Here we can't put "None" in the null values because a house must have "Electrical" items/fuses. So we will fill null values with "Sbrkr" in this column.

In [ ]:
data['Electrical'].fillna("Sbrkr", inplace=True)

##### Now we have to do this task for each columns with null/nan values (that's why I mentioned it as a tidius part).

In [ ]:
data['MSZoning'].value_counts()
#Filling null values with 'RL'
data['MSZoning'].fillna("RL",inplace=True)

#Filling nul values with mean
data['LotFrontage'].fillna(data['LotFrontage'].mean(), inplace=True)

data['Alley'].fillna("Nothing", inplace=True)
data['Utilities'].fillna("AllPub", inplace=True)
data['Exterior1st'].fillna("VinylSd", inplace=True)
data['Exterior2nd'].fillna("VinylSd", inplace=True)
data['MasVnrArea'].fillna(0, inplace=True)
data['MasVnrType'].fillna("None", inplace=True)
data['BsmtCond'].fillna("No", inplace=True)
data['BsmtExposure'].fillna("NB", inplace=True)
data['BsmtFinType1'].fillna("NB", inplace=True)
data['BsmtFinSF1'].fillna(0.0, inplace=True)
data['BsmtFinSF2'].fillna(0.0, inplace=True)
data['BsmtUnfSF'].fillna(0.0, inplace=True)
data['TotalBsmtSF'].fillna(0.0, inplace=True)
data['BsmtFullBath'].fillna(0.0, inplace=True)
data['BsmtHalfBath'].fillna(0.0, inplace=True)
data['KitchenQual'].fillna("TA", inplace=True)
data['Functional'].fillna("Typ", inplace=True)
data['FireplaceQu'].fillna("None", inplace=True)
data['GarageType'].fillna("No", inplace=True)
data['GarageYrBlt'].fillna(0, inplace=True)
data['GarageFinish'].fillna("No", inplace=True)
data['GarageCars'].fillna(0, inplace=True)
data['GarageArea'].fillna(0, inplace=True)
data['GarageQual'].fillna("No", inplace=True)
data['GarageCond'].fillna("No", inplace=True)
data['PoolQC'].fillna("No", inplace=True)
data['Fence'].fillna("No", inplace=True)
data['MiscFeature'].fillna("No", inplace=True)
data['SaleType'].fillna("Con", inplace=True)
data['SaleCondition'].fillna("None", inplace=True)
data['BsmtQual'].fillna("TA", inplace=True)
data['BsmtFinType2'].fillna("Unf", inplace=True)

##### Now let's see what is the number of null values.

In [ ]:
data.isnull().sum().sum()

> ##we have treated all the null values.

##### Let's deal with different types of data types in the dataset

In [ ]:
int_columns = data[data.columns[data.dtypes=='int']]
int_columns.columns

In [ ]:
data['MSZoning'].unique()

In [ ]:
object_columnns = data[data.columns[data.dtypes=='object']]
object_columnns.columns

In [ ]:
float_columns = data[data.columns[data.dtypes=='float']]
float_columns.columns

> ## Label Encoding the categorical variables

In [ ]:
from sklearn.preprocessing import LabelEncoder
for i in object_columnns:
    label = LabelEncoder()
    label.fit(data[i].values)
    data[i] = label.transform(data[i].values)

In [ ]:
object_columnns = data[data.columns[data.dtypes=='object']]
object_columnns.columns

In [ ]:
int_columns = data[data.columns[data.dtypes=='int']]
int_columns.columns

In [ ]:
data.head()

##### So now we can see that all the object columns are turned to int
#### Let's split back the train and test data 

In [ ]:
X_ = data[data.Type==1]
X_ = X_.drop(["Type"], axis=1)

test_ = data[data.Type==0]
test_ = test_.drop(["Type"], axis=1)

In [ ]:
X_.shape, y.shape, test_.shape 

## Scaling
##### It is required because dataset has columns which varies highly in magnitudes. If scaling is not performed then high magnitude values will have more impact on modelling.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
x_scaled = minmaxscaler.fit_transform(X_)

In [ ]:
#We can do Scaling directly with formula but we have pre-defined libraries so we will use them.. 
x_scaled_formula = X_.copy()
for cols in x_scaled_formula.columns:
    x_scaled_formula[cols] = x_scaled_formula[cols] / x_scaled_formula[cols].abs().max()

In [ ]:
x_scaled_formula.head()

> ## Modelling aka ML
##### There are so many regression algorithms which we can use, so we need to use most of them and then find out the best out of them.

##### Let's do the train_test_split first

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_, y, test_size=0.25, random_state=1)

* ###  Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_randomforest = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=13)
model_randomforest.fit(x_train, y_train)

In [ ]:
y_predict_randomforest = model_randomforest.predict(x_test)

In [ ]:
model_randomforest.score(x_test, y_test)

* ### XGBoost

In [ ]:
import xgboost as xgb
model_xgboost = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
 
model_xgboost.fit(x_train, y_train)

In [ ]:
model_xgboost.score(x_train, y_train)

In [ ]:
model_xgboost.score(x_test, y_test)

In [ ]:
y_predict_xgb = model_xgboost.predict(test_)

##### Let's try to submit the submission first.

In [ ]:
result = pd.DataFrame()
result['Id'] = test['Id']
result['SalePrice'] = y_predict_xgb

In [ ]:
result.head()